In [1]:
import os
from utils import predict
import seaborn as sns
from tensorflow.keras.models import load_model
from utils import prepare_training_data
from sklearn.preprocessing import StandardScaler
from utils import enrich_data
import matplotlib.pyplot as plt  # Import the matplotlib.pyplot module
import shutil  # Import the shutil module for file operations
import pandas as pd

# Define the path to the data folder
data_folder = 'data/predict'
output_folder = 'generated_plots'  # Specify the output folder

# Get a list of all files in the data folder
file_list = [filename for filename in os.listdir(data_folder) if filename.endswith('.csv')]

# Clean the output folder before each run
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)  # Remove the folder and its contents

# Create the output folder
os.makedirs(output_folder)

# Loop through each file
for filename in file_list:
    file_path = os.path.join(data_folder, filename)
    
    # Read the csv file
    df, labeled_data, df_for_training, cols = enrich_data(file_path)
    #print(labeled_data)
    
    # normalize the dataset
    scaler = StandardScaler()
    scaler = scaler.fit(df_for_training)
    df_for_training_scaled = scaler.transform(df_for_training)

    trainX, trainY = prepare_training_data(df_for_training_scaled)
    #print('trainX', trainX)
    #print('trainY', trainY)

    # Load model
    model = load_model('a_model')

    df_forecast = predict(model, df, trainX, trainY, df_for_training_scaled, scaler)
    #print('df_forecast', df_forecast)

    # Convert the filtered labeled_data list to a DataFrame
    original = pd.DataFrame(labeled_data, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'TrendLabel', 'HighLow', 'CloseOpen'])

    # Convert the 'Date' column to datetime
    original['Date'] = pd.to_datetime(original['Date'])

    # Filter original data to keep only dates after '2023-01-01'
    original = original[original['Date'] >= '2023-01-01']

    # Get the first date from df_forecast
    first_forecast_date = df_forecast['Date'].iloc[0]

    # Remove data from original DataFrame after the first forecast date
    original = original[original['Date'] <= first_forecast_date]

    # Create the Seaborn plot for 'CloseOpen'
    sns.lineplot(data=original, x='Date', y='CloseOpen')
    sns.lineplot(data=df_forecast, x='Date', y='CloseOpen')

    # Save the plot as an image file in the output folder
    output_filename_closeopen = os.path.join(output_folder, os.path.splitext(filename)[0] + '_close_open_plot.png')
    plt.savefig(output_filename_closeopen)

    # Clear the current figure
    plt.clf()

    # Create the Seaborn plot for 'Close'
    sns.lineplot(data=original, x='Date', y='Close')
    sns.lineplot(data=df_forecast, x='Date', y='Close')

    # Save the plot as an image file in the output folder
    output_filename_close = os.path.join(output_folder, os.path.splitext(filename)[0] + '_close_plot.png')
    plt.savefig(output_filename_close)

    # Clear the current figure
    plt.clf()

Predicting...
predict_period_dates 15 [Timestamp('2023-07-24 00:00:00'), Timestamp('2023-07-25 00:00:00'), Timestamp('2023-07-26 00:00:00'), Timestamp('2023-07-27 00:00:00'), Timestamp('2023-07-28 00:00:00'), Timestamp('2023-07-31 00:00:00'), Timestamp('2023-08-01 00:00:00'), Timestamp('2023-08-02 00:00:00'), Timestamp('2023-08-03 00:00:00'), Timestamp('2023-08-04 00:00:00'), Timestamp('2023-08-07 00:00:00'), Timestamp('2023-08-08 00:00:00'), Timestamp('2023-08-09 00:00:00'), Timestamp('2023-08-10 00:00:00'), Timestamp('2023-08-11 00:00:00')]
1/1 [==============================] - 0s 225ms/step
y_pred_future_close [1.27144153 1.27699035 1.27134301 1.26673392 1.27349202 1.27851529
 1.28504651 1.28311281 1.28430228 1.28995375 1.28897052 1.29848873
 1.29260377 1.29506766 1.29582068]
y_pred_future_closeopen [1.53832708 1.5385719  1.53843374 1.53835028 1.53856011 1.53860082
 1.5386326  1.53841885 1.53836052 1.53846822 1.53849293 1.53871413
 1.53852779 1.5385123  1.53858194]
Predicting...
pr

<Figure size 640x480 with 0 Axes>